In [1]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

# %run Model_experiment.ipynb
# %run Model_EncoderLstm.ipynb
%run Model_structure.ipynb
%run Dataset.ipynb

# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")

Finished.
Vocabulary sizes:
8315
6384


In [2]:
###Bleu
from nltk.translate.bleu_score import sentence_bleu

class BleuScore:
    def __init__(self):
        self.total_bleu=0
        self.total_num=0
    
    @staticmethod
    def cal_single_bleu(reference, candidate):
        return sentence_bleu(reference, candidate)
    
    def add(self,reference, candidate):
        self.total_bleu+=sentence_bleu(reference, candidate)
        self.total_num+=1
    
    def get_score(self):
        return (self.total_bleu/self.total_num)*100


In [3]:
def cal_batch_bleu(data_iter,model,batchsize,max_len=72,start_symbol=0,pad_idx=2,eos_string="</s>"):
    global vocab_src, vocab_tgt, spacy_de, spacy_en
    
    bleu = BleuScore()
    for batch in data_iter:
        memory = model.encode(batch.src, batch.src_mask)
        ys = torch.zeros(batchsize, 1).fill_(start_symbol).type_as(batch.src.data)
        for i in range(max_len - 1):
            #(batchsize, seq, 512)

            out = model.decode(memory, batch.src_mask, ys, subsequent_mask(ys.size(1)).type_as(batch.src.data))
            #(batchsize, vocabsize)
            prob = model.generator(out[:, -1, :])
            #(batchsize)
            _, next_word = torch.max(prob, dim=1)
            ys = torch.cat([ys, next_word.unsqueeze(1)], dim=1)
        for i in range(batchsize):
            src, tgt, pred = batch.src[i, :], batch.tgt[i, :], ys[i, :]
            src = [vocab_src.get_itos()[x] for x in src if x != pad_idx][1:-1]
            tgt = [vocab_tgt.get_itos()[x] for x in tgt if x != pad_idx][1:-1]
            pred = (" ".join([vocab_tgt.get_itos()[x] for x in pred if x != pad_idx]).split(eos_string, 1)[0]
                    + eos_string).split(' ')[1:-1]
            bleu.add([tgt], pred)
    print(bleu.total_num, bleu.get_score())
    return bleu.get_score()
#             print(BleuScore.cal_single_bleu([tgt],pred))
#     print(bleu.get_score())
            

In [4]:
def eval_worker(batchsize, max_padding=72):

    torch.cuda.set_device(0)
    pad_idx = vocab_tgt["<blank>"]
    d_model = 512
    model = make_model(len(vocab_src), len(vocab_tgt), N=6, d_model=512, d_ff=2048, h=4, dropout=0.1)
    model.cuda(0)
    model.load_state_dict(torch.load("./base/multi30k_best.pt"))
    module = model

    train_dataloader, valid_dataloader = create_dataloaders(
        0,
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=batchsize,
        max_padding=max_padding,
    )
    
    
    model.eval()
    cal_batch_bleu((Batch(b[0], b[1], pad_idx) for b in valid_dataloader), model, batchsize)
    torch.cuda.empty_cache()

In [5]:
eval_worker(batchsize=35)

1015 27.632053646214967
